In [1]:
import random
import numpy as np

# DataModel

In [20]:
class Player(object):
    def __init__(self, name, stats):
        self.name = name
        self.stats = stats
    
class Stats(object):
    def __init__(self, points, assists, rebounds):
        self.points = points
        self.assists = assists
    #self.steals = 0
    #self.tos = 0
        self.rebounds = rebounds
    #self.fgp = 0
    #self.ftm = 0
    #self.tpm = 0
    #self.blocks = 0
    #self.ftp = 0

In [21]:
class Agent(object):
    
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def action(self, environment):
        pass

class RandomAgent(Agent):
    def __init__(self, players = []):
        self.players = players
        
    def action(self, environment):
        envPlayers = environment.players
        selected = envPlayers.pop(random.randint(0,len(envPlayers)-1))
        self.players.append(selected)
        return Environment(players = envPlayers, agents = environment.agents)
    
class MaxPointsAgent(Agent):
    def __init__(self, players = []):
        self.players = players
        
    def action(self, environment):
        envPlayers = environment.players
        points_vec = [player.stats.points for player in players]
        i = np.argmax(points_vec)
        selected = envPlayers.pop(i)
        self.players.append(selected)
        return Environment(players = envPlayers, agents = environment.agents)
    
class MonteCarloAgent(Agent):
    def __init__(self, scorer, players = []):
        self.players = players
        self.scorer = scorer
    
    def runSimulation(self, env):
        return env
    
    def action(self, environment):
        player_tracker = {}
        newEnv = runSimulation(environment)
        winner = scorer.score(newEnv.agents)
        winningAgent = agents[winner]
        for player in winningAgent.players:
            player_tracker.update({player: int(player_tracker.get(player) or 0) + 1})
        winningPlayer = player_tracker[np.argmax(player_tracker.values)]
#do something once winning
        return Environment(players = envPlayers, agents = environment.agents)

In [28]:
class Scorer(object):
    
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def score(self, agents):
        pass

class SimpleSingleClassScorer(Scorer):
    """
        Takes a list of agents. 
    
        Scores by taking the argmax of a single category for those agents. 
        
        Returns index of winner
    """
    def score(self, agents):
        list_of_points = []
        for agent in agents:
            players = agent.players
            total_points = sum([player.stats.points for player in players])
            list_of_points.append(total_points)
        return np.argmax(list_of_points)
    
class SimpleMultiClassScorer(Scorer):
    """
        Takes a list of agents. 
    
        Scores by taking the argmax of a single category for those agents. 
        
        Returns a vector in the order of the agents, representing
        the commulative score of each agent
    """
    
    def __init__(self):
        self.keys = ['points', 'assists']
        
    def score(self, agents):
        list_of_points = []
        list_of_assists = []
        list_of_rebounds = []
        score_dict = {}
        for (i, agent) in enumerate(agents):
            score_dict.update({i:0})
            players = agent.players
            total_points = sum([player.stats.points for player in players])
            total_assists = sum([player.stats.assists for player in players])
            total_rebounds = sum([player.stats.rebounds for player in players])
            list_of_points.append(total_points)
            list_of_assists.append(total_assists)
            list_of_rebounds.append(total_rebounds)
        index_points = np.argmax(list_of_points)
        index_assists = np.argmax(list_of_assists)
        index_rebounds = np.argmax(list_of_rebounds)
        score_dict.update({index_points: int(score_dict.get(index_points) or 0) + 1})
        score_dict.update({index_assists: int(score_dict.get(index_assists) or 0) + 1})
        score_dict.update({index_rebounds: int(score_dict.get(index_rebounds) or 0) + 1})
        i = np.argmax(score_dict.values())
        return i

In [37]:
from abc import ABCMeta, abstractmethod

class Environment(object):
    def __init__(self, players, agents):
        self.players = players
        self.agents = agents
        
class Game(object):
    def __init__(self, players, agents, scorer):
        self.environment = Environment(players = players, agents = agents)
        self.agents = agents
        self.scorer = scorer
        
    def run(self, numberOfDraftRounds):
        newEnv = self.draft(self.environment, numberOfDraftRounds)
        finalAgents = newEnv.agents
        i = scorer.score(finalAgents)
        return (finalAgents[i], [player.name for player in finalAgents[i].players])
                       
    def draft(self, env, numberOfDraftRounds):
        newEnv = env
        for round in range(0, numberOfDraftRounds):
            for agent in env.agents:
                newEnv = agent.action(newEnv)
        return newEnv
            

# Experiment

In [42]:
p1 = Player(name = 'p1', stats = Stats(points = 10, assists = 7, rebounds = 15))
p2 = Player(name = 'p2', stats = Stats(points = 3, assists = 4, rebounds = 7))
p3 = Player(name = 'p3', stats = Stats(points = 15, assists = 10, rebounds = 20))


a1 = RandomAgent(players = [])
a2 = RandomAgent(players = [])
a3 = MaxPointsAgent(players = [])

players = [p1,p2,p3]
agents = [a1, a2, a3]
scorer = SimpleMultiClassScorer()

game = Game(players = players, agents = agents, scorer = scorer)

game.run(1)

(<__main__.RandomAgent at 0x10fe5f0d0>, ['p3'])

# ScoreAndPick

# ComboSolver

# MonteCarloSortAndPick